In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train_balanced.csv')
test = pd.read_csv('./data/test_balanced.csv')
len(train)

FileNotFoundError: [Errno 2] No such file or directory: './data/test_balanced.csv'

In [8]:
df = pd.read_csv('./data/df_cleaned.csv')
df_grouped = df.groupby('star_rating').count()
min_ratings = df_grouped['review_body'].min()
reduced_number = 50000

df_for_testing = df[:20000]
df_for_training = df[20000:]

df_r1 = df_for_training[df_for_training['star_rating'] == 1].sample(n = reduced_number, random_state=1)
df_r2 = df_for_training[df_for_training['star_rating'] == 2].sample(n = reduced_number, random_state=1)
df_r3 = df_for_training[df_for_training['star_rating'] == 3].sample(n = reduced_number, random_state=1)
df_r4 = df_for_training[df_for_training['star_rating'] == 4].sample(n = reduced_number, random_state=1)
df_r5 = df_for_training[df_for_training['star_rating'] == 5].sample(n = reduced_number, random_state=1)

assert(len(df_r1) == len(df_r2) == len(df_r3) == len(df_r4) == len(df_r5))

df_train_balanced = pd.concat([df_r1, df_r2, df_r3, df_r4, df_r5])
train_balanced = df_train_balanced.sample(frac=1, random_state=1)

train_balanced.to_csv('./data/train_balanced_50000.csv',  index=False)
#df_for_testing.to_csv('./data/testing.csv',  index=False)

In [4]:
t = pd.read_csv('./data/train_balanced.csv')
print(len(t))

125000


In [9]:
t = pd.read_csv('./data/train_balanced_50000.csv')
print(len(t))

250000
